# Purpose
This notebook details data preparation and modelling of the most recent model iteration.

1. Feature Engineering

2. Modelling

In [8]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from sqlalchemy import create_engine
from src import local
from src import functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Feature engineering
### Features
The model in this notebook uses differentials and per-minute rates for the following technique categories:
- Significant Strikes
    - Total
    - on the Ground
    - in the Clinch
    - at a Distance
- takedowns

#### Differentials
Differentials are the difference in one fighters metric and his opponents. In this model, the differentials are calculated from
the per-15-minute or per-1-minute rates in a single round.

### Target
The target is the Combined Significant Strike Attempts per-1-minute of both fighters, which allows us to measure the amount of striking action in a fight.

#### Load the data

In [9]:
# Set up tables
# Credentials
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')

#### Join our 5 advanced statistics tables with our bouts and events table to get dates for each bout

In [10]:
query = """
SELECT striking_adv.bout_id, striking_adv.fighter_id, striking_adv.round, date, 
ss_a_p15m, td_s_p15m_di, ss_s_p15m_di, ss_a_p15m_di, 
g_ss_s_p15m_di, g_ss_a_p15m_di, c_ss_s_p15m_di,
c_ss_a_p15m_di, d_ss_a_p15m_di, d_ss_s_p15m_di
FROM striking_adv
    JOIN striking_position_adv ON 
    CONCAT(striking_adv.bout_id, striking_adv.fighter_id, CAST(striking_adv.round AS CHAR)) =
    CONCAT(striking_position_adv.bout_id, striking_position_adv.fighter_id, CAST(striking_position_adv.round AS CHAR))
    
    JOIN grappling_adv ON 
    CONCAT(striking_adv.bout_id, striking_adv.fighter_id, CAST(striking_adv.round AS CHAR)) =
    CONCAT(grappling_adv.bout_id, grappling_adv.fighter_id, CAST(grappling_adv.round AS CHAR))

JOIN bouts ON bouts.id = striking_adv.bout_id
JOIN events ON events.id = bouts.event_id
"""

data = pd.read_sql(query, engine)

In [11]:
data.head()

,bout_id,fighter_id,round,date,ss_a_p15m,td_s_p15m_di,ss_s_p15m_di,ss_a_p15m_di,g_ss_s_p15m_di,g_ss_a_p15m_di,c_ss_s_p15m_di,c_ss_a_p15m_di,d_ss_a_p15m_di,d_ss_s_p15m_di
0,000da3152b7b5ab1,6da99156486ed6c2,1,"July 08, 2006",117.0,3.0,21.0,84.0,0.0,3.0,9.0,27.0,54.0,12.0
1,000da3152b7b5ab1,6da99156486ed6c2,2,"July 08, 2006",78.0,0.0,18.0,21.0,-12.0,-18.0,21.0,27.0,12.0,9.0
2,000da3152b7b5ab1,6da99156486ed6c2,3,"July 08, 2006",69.0,6.0,9.0,42.0,6.0,18.0,3.0,3.0,21.0,0.0
3,000da3152b7b5ab1,d1a1314976c50bef,1,"July 08, 2006",33.0,-3.0,-21.0,-84.0,0.0,-3.0,-9.0,-27.0,-54.0,-12.0
4,000da3152b7b5ab1,d1a1314976c50bef,2,"July 08, 2006",57.0,0.0,-18.0,-21.0,12.0,18.0,-21.0,-27.0,-12.0,-9.0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25772 entries, 0 to 25771
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bout_id         25772 non-null  object 
 1   fighter_id      25772 non-null  object 
 2   round           25772 non-null  object 
 3   date            25772 non-null  object 
 4   ss_a_p15m       25772 non-null  float64
 5   td_s_p15m_di    25772 non-null  float64
 6   ss_s_p15m_di    25772 non-null  float64
 7   ss_a_p15m_di    25772 non-null  float64
 8   g_ss_s_p15m_di  25772 non-null  float64
 9   g_ss_a_p15m_di  25772 non-null  float64
 10  c_ss_s_p15m_di  25772 non-null  float64
 11  c_ss_a_p15m_di  25772 non-null  float64
 12  d_ss_a_p15m_di  25772 non-null  float64
 13  d_ss_s_p15m_di  25772 non-null  float64
dtypes: float64(10), object(4)
memory usage: 2.8+ MB


In [13]:
data['date'] = pd.to_datetime(data['date'])

## Create fighter-bout instance dataframe

A fighter-bout instance represents one fighter in one bout.
 - The same fighter has exactly one fighter-bout instance for every single bout he has been in. 
 - Every bout has exactly two fighter-bout instances, one for each fighter in the bout. 
  
In this case a fighter-bout instance is assigned a unique identifier comprised of the bout_id combined with the fighter_link.

In [14]:
fighter_bout_inst = functions.create_fighter_bout_instance_table(data, 'ss_a_p15m')

In [15]:
fighter_bout_inst

,bout_id,fighter_id,date,target
fighter_bout_inst,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,88.000000
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,39.000000
0019ec81fd706ade326f94d6cfb1bf25,0019ec81fd706ade,326f94d6cfb1bf25,2019-10-18,97.000000
0019ec81fd706ade85073dbd1be65ed9,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,105.000000
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,38.250000
...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0.000000
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,217.834395
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,114.649682


#### Remove bouts that take place after 2012

Mixed Martial Arts has changed over the years and stylistic match ups from a decade ago might not be analgous to matchups today.
Almost every fighter nowadays has at least some exposure to Wrestling, Brazilian Jiu Jitsu, Boxing, and Muay Thai, so stylistic 
differences tend to reflect personal choices by the fighter rather than the limits of their martial arts discipline. For now,
we choose 2012 as the cut off point, but further analysis needs to be done regarding the appropriate date.

In [16]:
fighter_bout_inst = fighter_bout_inst[fighter_bout_inst['date']>pd.to_datetime('12-31-2011')]

In [17]:
fighter_bout_inst

,bout_id,fighter_id,date,target
fighter_bout_inst,,,,
0019ec81fd706ade326f94d6cfb1bf25,0019ec81fd706ade,326f94d6cfb1bf25,2019-10-18,97.000000
0019ec81fd706ade85073dbd1be65ed9,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,105.000000
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,38.250000
0027e179b743c86c91ea901c458e95dd,0027e179b743c86c,91ea901c458e95dd,2015-03-14,51.187500
002921976d27b7dab4ad3a06ee4d660c,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,88.932806
...,...,...,...,...
ffd3e3d37cba32da92a9aa9c93192871,ffd3e3d37cba32da,92a9aa9c93192871,2014-10-25,154.000000
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,217.834395
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,114.649682


## Calculate metrics

The metrics will be calculated from the fighters entire career and from their last 3 fights, to give an idea of their current state.
Career averages are prefixed with 'ca_' and 3 fight averages with '3fa_'

In [18]:
list_of_metrics = ['ss_a_p15m', 'td_s_p15m_di', 'ss_s_p15m_di', 'ss_a_p15m_di', 
                    'g_ss_s_p15m_di', 'g_ss_a_p15m_di', 'c_ss_s_p15m_di',
                    'c_ss_a_p15m_di', 'd_ss_a_p15m_di', 'd_ss_s_p15m_di']

#### How these metrics are calculated
The following cell iterates through each row in the fighter bout instance table. It takes the unique fighter_id and the date from that row and calculates the fighters metrics up until that date. This represents the fact that our model will only have prior knowledge of the fighters when making it's predictions.


This cell takes about 5-10 minutes to load.

In [19]:
for metric in list_of_metrics:
    print(metric+'\n')
    values = fighter_bout_inst.apply(lambda row: functions.calculate_metric_average(metric, 
                                                                                    row['fighter_id'], 
                                                                                    row['date'], 
                                                                                    data), 
                                     axis=1)

    fighter_bout_inst['ca_'+metric] = values.map(lambda x: x[0])
    fighter_bout_inst['3fa_'+metric] = values.map(lambda x: x[1])

ss_a_p15m



<ipython-input-19-bedfd297c482>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['ca_'+metric] = values.map(lambda x: x[0])
<ipython-input-19-bedfd297c482>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['3fa_'+metric] = values.map(lambda x: x[1])


td_s_p15m_di

ss_s_p15m_di

ss_a_p15m_di

g_ss_s_p15m_di

g_ss_a_p15m_di

c_ss_s_p15m_di

c_ss_a_p15m_di

d_ss_a_p15m_di

d_ss_s_p15m_di



### Debut fights and inexperienced fighters
Many fights include fighter who have never fought in the UFC before, and some do not have long enough records to calculate 3 fight averages. These fighters would have null values in their career and 3-fight-averages, so I drop all of those rows here.

In [20]:
mask = fighter_bout_inst['3fa_ss_a_p15m'].notnull()
fighter_bout_inst = fighter_bout_inst[mask]

In [21]:
fighter_bout_inst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6069 entries, 0019ec81fd706ade85073dbd1be65ed9 to fffa21388cdd78b7c80095f6092271a7
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   bout_id             6069 non-null   object        
 1   fighter_id          6069 non-null   object        
 2   date                6069 non-null   datetime64[ns]
 3   target              6069 non-null   float64       
 4   ca_ss_a_p15m        6069 non-null   float64       
 5   3fa_ss_a_p15m       6069 non-null   object        
 6   ca_td_s_p15m_di     6069 non-null   float64       
 7   3fa_td_s_p15m_di    6069 non-null   object        
 8   ca_ss_s_p15m_di     6069 non-null   float64       
 9   3fa_ss_s_p15m_di    6069 non-null   object        
 10  ca_ss_a_p15m_di     6069 non-null   float64       
 11  3fa_ss_a_p15m_di    6069 non-null   object        
 12  ca_g_ss_s_p15m_di   6069 non-null   float64       

## Create the final dataframe

The current fighter bout instance table has two rows for each fight (one row for each fighter). In order to create a table where each row represents one fight, I need get each fighter on the same row.

In [22]:
model_df = functions.merge_fighter_instances(fighter_bout_inst)

/home/max/sports_science/match_finder/src/functions.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instances_df['inst_id'] = instances_df['bout_id'] + instances_df['fighter_id']


## Creating Combined Significant Strike Attempts Per 15 Minute (c_sig_str_a_p15m)

In [23]:
model_df['c_ss_a_p15m'] = model_df['target_0'] + model_df['target_1']
model_df

,bout_id,fighter_id_0,date_0,target_0,ca_ss_a_p15m_0,3fa_ss_a_p15m_0,ca_td_s_p15m_di_0,3fa_td_s_p15m_di_0,ca_ss_s_p15m_di_0,3fa_ss_s_p15m_di_0,...,ca_c_ss_s_p15m_di_1,3fa_c_ss_s_p15m_di_1,ca_c_ss_a_p15m_di_1,3fa_c_ss_a_p15m_di_1,ca_d_ss_a_p15m_di_1,3fa_d_ss_a_p15m_di_1,ca_d_ss_s_p15m_di_1,3fa_d_ss_s_p15m_di_1,inst_id_1,c_ss_a_p15m
0,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,105.000000,307.256757,307.257,-0.750000,-0.75,201.727477,201.727,...,7.853604,7.8536,7.853604,7.8536,43.099099,43.0991,22.520270,22.5203,0019ec81fd706ade85073dbd1be65ed9,210.000000
1,0027e179b743c86c,91ea901c458e95dd,2015-03-14,51.187500,78.528358,67.1834,1.500000,1.71429,23.986567,27.838,...,-5.000000,-5,-3.000000,-3,-86.000000,-86,-38.000000,-38,0027e179b743c86c3aa794cbe1e3484b,89.437500
2,002921976d27b7da,ebc1f40e00e0c481,2014-12-13,17.786561,131.820971,136.544,-0.433461,-1,-13.753972,-21.3637,...,3.825798,22.1577,-7.405384,30.1415,-46.427974,-12.943,-18.981288,11.4666,002921976d27b7dab4ad3a06ee4d660c,106.719368
3,002c1562708ac307,44470bfd9483c7ad,2014-05-24,160.975610,48.000000,48,3.000000,3,7.000000,7,...,2.628510,1.42347,3.107143,2.70918,27.566239,36.3214,3.747558,-3.9949,002c1562708ac30722a92d7f62195791,406.097561
4,003c84fc7c3fa028,49858bf46dabf6fb,2019-05-04,201.000000,95.199683,114.61,1.278884,0,13.101610,-20.051,...,0.541620,-4.38289,-0.526447,-5.5671,-24.702311,-61.0355,7.922476,1.017,003c84fc7c3fa02849858bf46dabf6fb,402.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,ff64fc34065565d0,6fb1ba67bef41b37,2015-05-30,92.000000,97.293860,104.333,1.429825,-0.333333,8.600877,10.3333,...,5.199375,8.58364,6.996808,11.8649,-9.012294,-10.3594,-0.139091,-4.54688,ff64fc34065565d007225ba28ae309b6,208.857143
3535,ffbc12e4f821ec68,3591d0d5d382a381,2014-02-15,71.000000,72.673128,74.3333,2.051471,1.66667,20.581217,5.33333,...,4.505682,2.33333,3.755682,1.66667,-1.670789,0,-2.784759,-4,ffbc12e4f821ec683591d0d5d382a381,142.000000
3536,ffd3e3d37cba32da,92a9aa9c93192871,2014-10-25,154.000000,143.855810,175.261,3.018444,0,4.804949,-10.3109,...,-2.625000,-3,-3.750000,-4.28571,6.414204,11.3992,-11.206462,-6.29744,ffd3e3d37cba32da7413b80dbb0f8f9f,236.000000
3537,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,114.649682,88.000347,98,3.846692,3.66667,-6.478311,-12.3333,...,1.852669,0.688907,1.276528,-1.46939,16.600578,0.152086,7.633712,13.4629,ffea776913451b6d22a92d7f62195791,332.484076


#### drop unnecessary columns

In [24]:
model_df.columns

Index(['bout_id', 'fighter_id_0', 'date_0', 'target_0', 'ca_ss_a_p15m_0',
       '3fa_ss_a_p15m_0', 'ca_td_s_p15m_di_0', '3fa_td_s_p15m_di_0',
       'ca_ss_s_p15m_di_0', '3fa_ss_s_p15m_di_0', 'ca_ss_a_p15m_di_0',
       '3fa_ss_a_p15m_di_0', 'ca_g_ss_s_p15m_di_0', '3fa_g_ss_s_p15m_di_0',
       'ca_g_ss_a_p15m_di_0', '3fa_g_ss_a_p15m_di_0', 'ca_c_ss_s_p15m_di_0',
       '3fa_c_ss_s_p15m_di_0', 'ca_c_ss_a_p15m_di_0', '3fa_c_ss_a_p15m_di_0',
       'ca_d_ss_a_p15m_di_0', '3fa_d_ss_a_p15m_di_0', 'ca_d_ss_s_p15m_di_0',
       '3fa_d_ss_s_p15m_di_0', 'inst_id_0', 'fighter_id_1', 'date_1',
       'target_1', 'ca_ss_a_p15m_1', '3fa_ss_a_p15m_1', 'ca_td_s_p15m_di_1',
       '3fa_td_s_p15m_di_1', 'ca_ss_s_p15m_di_1', '3fa_ss_s_p15m_di_1',
       'ca_ss_a_p15m_di_1', '3fa_ss_a_p15m_di_1', 'ca_g_ss_s_p15m_di_1',
       '3fa_g_ss_s_p15m_di_1', 'ca_g_ss_a_p15m_di_1', '3fa_g_ss_a_p15m_di_1',
       'ca_c_ss_s_p15m_di_1', '3fa_c_ss_s_p15m_di_1', 'ca_c_ss_a_p15m_di_1',
       '3fa_c_ss_a_p15m_di_1', 

In [25]:
model_df.loc[0]

bout_id                                 0019ec81fd706ade
fighter_id_0                            85073dbd1be65ed9
date_0                               2019-10-18 00:00:00
target_0                                             105
ca_ss_a_p15m_0                                   307.257
3fa_ss_a_p15m_0                                  307.257
ca_td_s_p15m_di_0                                  -0.75
3fa_td_s_p15m_di_0                                 -0.75
ca_ss_s_p15m_di_0                                201.727
3fa_ss_s_p15m_di_0                               201.727
ca_ss_a_p15m_di_0                                233.973
3fa_ss_a_p15m_di_0                               233.973
ca_g_ss_s_p15m_di_0                              171.354
3fa_g_ss_s_p15m_di_0                             171.354
ca_g_ss_a_p15m_di_0                               183.02
3fa_g_ss_a_p15m_di_0                              183.02
ca_c_ss_s_p15m_di_0                               7.8536
3fa_c_ss_s_p15m_di_0           

In [26]:
model_df.fighter_id_0

0       85073dbd1be65ed9
1       91ea901c458e95dd
2       ebc1f40e00e0c481
3       44470bfd9483c7ad
4       49858bf46dabf6fb
              ...       
3534    6fb1ba67bef41b37
3535    3591d0d5d382a381
3536    92a9aa9c93192871
3537    75e5fec9f72910ef
3538    c80095f6092271a7
Name: fighter_id_0, Length: 3539, dtype: object

In [27]:
mask = model_df.apply(lambda row: row['fighter_id_0'] == row['fighter_id_1'], axis=1)
model_df[mask].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 3535
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   bout_id               1009 non-null   object        
 1   fighter_id_0          1009 non-null   object        
 2   date_0                1009 non-null   datetime64[ns]
 3   target_0              1009 non-null   float64       
 4   ca_ss_a_p15m_0        1009 non-null   float64       
 5   3fa_ss_a_p15m_0       1009 non-null   object        
 6   ca_td_s_p15m_di_0     1009 non-null   float64       
 7   3fa_td_s_p15m_di_0    1009 non-null   object        
 8   ca_ss_s_p15m_di_0     1009 non-null   float64       
 9   3fa_ss_s_p15m_di_0    1009 non-null   object        
 10  ca_ss_a_p15m_di_0     1009 non-null   float64       
 11  3fa_ss_a_p15m_di_0    1009 non-null   object        
 12  ca_g_ss_s_p15m_di_0   1009 non-null   float64       
 13  3fa_g_ss_s_p15m_di

In [28]:
def merge_fighter_instances(instances_df, flip=False, rounds=False):
    """
    input: instance table where each row represents one fighter in one bout (in one round, if round instances)
    output: table where each row represents both fighters in a single bout (or round, if round instances)
    
    example input
    fighter_bout_instance_table:
    name:     |bout: 
    fighter_0 |1
    fighter_1 |1
    fighter_0 |2
    fighter_1 |2
    
    example output
    bout_table
    name_0:   |name_1:    |bout: 
    fighter_0 |fighter_1  |1
    fighter_0 |fighter_1  |2
    """
    # set the primary key for the rows to be joined on
    if rounds:
        instances_df['round_id'] = instances_df['bout_id']+instances_df['round']
        key = 'round_id'
    else:
        key = 'bout_id'
    
    # identifier for each fighter in each bout
    instances_df['inst_id'] = instances_df['bout_id'] + instances_df['fighter_id']
    
    # Create two lists of instance IDs, 
    # both list has one fighter per bout, 
    # both lists are unique.
    # if they are concatenated together they would 
    # create the original list of fighter instances
    fighter_0 = list(instances_df.groupby(key).inst_id.max())
    print('------------------------------------------------\n', fighter_0)
    fighter_1 = list(instances_df.groupby(key).inst_id.min())
    print(fighter_1, '\n------------------------------------------------')
        
    # filter the instances dataframe using the lists above
    mask = instances_df['inst_id'].isin(fighter_0)
    instances_df_0 = instances_df[mask]
    instances_df_0

    mask = instances_df['inst_id'].isin(fighter_1)
    instances_df_1 = instances_df[mask]
    instances_df_1
    
    # Used for creating advanced statistics. reverses the order of the fighters
    if flip:
        model_df = pd.merge(instances_df_1, instances_df_0, on=key, suffixes=('_0', '_1'))
        return model_df
    else:
        model_df = pd.merge(instances_df_0, instances_df_1, on=key, suffixes=('_0', '_1'))
        return model_df
    

In [49]:
fighter_bout_inst['inst_id'] = fighter_bout_inst['bout_id'] + fighter_bout_inst['fighter_id']

# Create two lists of instance IDs, 
# both list has one fighter per bout, 
# both lists are unique.
# if they are concatenated together they would 
# create the original list of fighter instances
fighter_0 = list(fighter_bout_inst.groupby('bout_id').inst_id.max())[0]
print('------------------------------------------------\n', fighter_0)
fighter_1 = list(fighter_bout_inst.groupby('bout_id').inst_id.min())[0]
print(fighter_1, '\n------------------------------------------------')

<ipython-input-49-548cef3d98cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['inst_id'] = fighter_bout_inst['bout_id'] + fighter_bout_inst['fighter_id']


------------------------------------------------
 0019ec81fd706ade85073dbd1be65ed9
0019ec81fd706ade85073dbd1be65ed9 
------------------------------------------------


bout_id
0019ec81fd706ade    0019ec81fd706ade85073dbd1be65ed9
0027e179b743c86c    0027e179b743c86c3aa794cbe1e3484b
002921976d27b7da    002921976d27b7dab4ad3a06ee4d660c
002c1562708ac307    002c1562708ac30722a92d7f62195791
003c84fc7c3fa028    003c84fc7c3fa02849858bf46dabf6fb
                                  ...               
ff64fc34065565d0    ff64fc34065565d007225ba28ae309b6
ffbc12e4f821ec68    ffbc12e4f821ec683591d0d5d382a381
ffd3e3d37cba32da    ffd3e3d37cba32da7413b80dbb0f8f9f
ffea776913451b6d    ffea776913451b6d22a92d7f62195791
fffa21388cdd78b7    fffa21388cdd78b75d7bdab5e03e3216
Name: inst_id, Length: 3539, dtype: object